# TensorFlow integration

HCIPy supports not only Numpy as its numerical backend, but also TensorFlow. This allows for using GPU acceleration as well as automatic differentiation for fast gradient evaluation. In this tutorial, we will propagate through a few optical systems with Tensorflow and perform gradient descent optimization.

In [ ]:
from hcipy import *
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Basics

In [ ]:
pupil_grid = make_pupil_grid(512)
focal_grid = make_focal_grid(8, 16)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

In [ ]:
# Numpy
aperture_np = circular_aperture(1)(pupil_grid)

# TensorFlow
aperture_tf = aperture_np.as_backend('tensorflow')

In [ ]:
print(type(aperture_np).__name__)
print(type(aperture_tf).__name__)

print(aperture_np)
print(aperture_tf)

In [ ]:
# Numpy
img_np = prop(Wavefront(aperture_np))

# TensorFlow
img_tf = prop(Wavefront(aperture_tf))

In [ ]:
plt.subplot(1,2,1)
imshow_psf(img_np, normalization='peak', title='Numpy')
plt.subplot(1,2,2)
imshow_psf(img_tf, normalization='peak', title='TensorFlow')
plt.show()

In [ ]:
influence_functions = make_xinetics_influence_functions(pupil_grid, 20, 1 / 20).to_dense()
dm = DeformableMirror(influence_functions)

In [ ]:
dm.random(0.03)
dm.tf_actuators.assign(dm.actuators)

# Numpy
img_np = prop(dm(Wavefront(aperture_np)))

# TensorFlow
img_tf = prop(dm(Wavefront(aperture_tf)))

In [ ]:
plt.subplot(1,2,1)
imshow_psf(img_np, normalization='peak', title='Numpy')
plt.subplot(1,2,2)
imshow_psf(img_tf, normalization='peak', title='TensorFlow')
plt.show()

In [ ]:
lyot_stop = circular_aperture(0.95)(pupil_grid)
coro = VortexCoronagraph(pupil_grid, 2, lyot_stop=lyot_stop)

# Numpy
coro_img_np = prop(coro(dm(Wavefront(aperture_np))))

# TensorFlow
coro_img_tf = prop(coro(dm(Wavefront(aperture_tf))))

In [ ]:
plt.subplot(1,2,1)
imshow_psf(coro_img_np, normalization=img_np.power.max(), title='Numpy')
plt.subplot(1,2,2)
imshow_psf(coro_img_tf, normalization=img_tf.power.max(), title='TensorFlow')
plt.show()

In [ ]:
print(type(np.abs(aperture_tf)**2))